In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from tokenizer import GECTokenizer

In [18]:
from utils import Dataset
from transformers import AutoTokenizer
from rules import Rule, create_rules, annotate_data_with_rules
import numpy as np
from subwords_aligner import Aligner
import re
import json

aligner = Aligner()

In [4]:
tokenizer = GECTokenizer('/scratch/ba63/BERT_models/bert-base-arabic-camelbert-msa')

In [5]:
train_dataset = Dataset('/scratch/ba63/gec/data/alignment/modeling_areta_tags_improved/'\
                        'qalb14/qalb14_train.areta+.txt')

tune_dataset =  Dataset('/scratch/ba63/gec/data/alignment/modeling_areta_tags_improved/'\
                        'qalb14/qalb14_tune.areta+.txt')

In [7]:
rules = create_rules(train_dataset, tokenizer)

Creating rules...
Done!
There are 5060 unique rules!


In [9]:
# cnt = 0
# for rule in rules:
#     if rule[1] >= 100:
#         cnt += 1
# cnt

In [32]:
rules_ = [x[0] for x in rules]

In [ ]:
annotated_data = annotate_data_with_rules(train_dataset, rules_)

In [15]:
annotated_data_tune = annotate_data_with_rules(dataset=tune_dataset, tokenizer=tokenizer, rules=rules_)

In [178]:
fixed_data = []
unk_cnt = 0

for sentence in annotated_data_tune:
    fixed_example = []

    for subword in sentence:
        rule = Rule.from_str(subword['rule'])

        if rule.edits != [['UNK']]:

            if subword['subword_model'].startswith('##'):
                fixed_example.append(f"##{rule.apply(subword['subword'])}")
                
            elif subword['subword'].startswith(' '):
                fixed_example.append(f" {rule.apply(subword['subword'])}")
                
            else:
                fixed_example.append(rule.apply(subword['subword']))

        else:
            unk_cnt += 1
            if subword['subword_model'].startswith('##'):
                fixed_example.append(f"##{subword['subword']}")
                
            elif subword['subword'].startswith(' '):
                fixed_example.append(f" {subword['subword']}")
            
            else:
                fixed_example.append(subword['subword'])

    fixed_data.append(fixed_example)

print(unk_cnt)

1053


In [30]:
def save_data(data, path):
    with open(path, 'w') as f:
        for ex in data:
            for x in ex:
                f.write(f"{x['subword_model']}\t{x['rule']}\n")
            f.write('\n')

In [31]:
save_data(annotated_data_tune, 'xyz.txt')

In [179]:
def detokenize(seq):
    detokenize = []
    for token in seq:
        if token.startswith('##'):
            detokenize[-1] += token.replace('##','')
        
        elif token.startswith(' '):
            detokenize[-1] += token.strip()
    
        else:
            detokenize.append(token)
    
    detokenize = ' '.join(detokenize)
    detokenize = re.sub(' +', ' ', detokenize)
    return detokenize

In [180]:
detokenized_data = [detokenize(seq) for seq in fixed_data]

In [181]:
with open('qalb14_tunes.preds.100.txt', 'w') as f:
    f.write('\n'.join(detokenized_data))
    f.write('\n')